In [1]:
!pip install -q gradio tensorflow torch timm pillow


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [4]:
import gradio as gr
import tensorflow as tf
import torch
import timm
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

# ================= CLASSES =================
CLASSES = ["Benign (İyi)", "Malignant (Kötü)", "Normal"]

# ================= CNN INPUT SIZES =================
TF_INPUT_SIZES = {
    "ResNet-50": (224,224),
    "DenseNet": (224,224),
    "EfficientNet": (224,224),
    "Inception": (299,299),
    "Xception": (224,224),
    "NasNet": (331,331),
}

# ================= TRANSFORMER REGISTRY =================
TORCH_MODELS = {
    "BEiT": {"arch": "beit_base_patch16_224", "mean":[0.5]*3, "std":[0.5]*3},
    "DeiT-Tiny": {"arch": "deit_tiny_patch16_224", "mean":[0.485,0.456,0.406], "std":[0.229,0.224,0.225]},
    "ViT (Standard)": {"arch": "vit_tiny_patch16_224", "mean":[0.485,0.456,0.406], "std":[0.229,0.224,0.225]},
    "Swin Transformer": {"arch": "swin_tiny_patch4_window7_224", "mean":[0.485,0.456,0.406], "std":[0.229,0.224,0.225]},
    "CAiT": {"arch": "cait_s24_224", "mean":[0.485,0.456,0.406], "std":[0.229,0.224,0.225]},
}




# ================= MODEL REGISTRY =================
MODEL_REGISTRY = {
    "BUSI - CNN": {
        "ResNet-50": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/ResNet_Results/Resnet_model_with_BUSI(-3).keras",
            "1e-4": "/content/drive/MyDrive/Ara Proje/saved_models/ResNet_Results/Resnet_model_with_BUSI(-4).keras",
            "1e-5": "/content/drive/MyDrive/Ara Proje/saved_models/ResNet_Results/Resnet_model_with_BUSI(-5).keras",
        },
        "EfficientNet": {
            "1e-4": "/content/drive/MyDrive/Ara Proje/saved_models/EfficientNet_Results/EfficientNet_best_lr_1en04.keras",
        },
        "NasNet": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/Nasnet_Results/nasnet_model_with_BUSI.keras",
        },
        "Inception": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/InceptionV3_Results/InceptionV3_best_lr_1en03.keras",
        },
        "Xception": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/Xception_Results/xception_model_with_BUSI.keras",
        },
        "DenseNet": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/DenseNet_Results/DenseNet121_best_lr_1en03.keras",
        }
    },

    "BUS-UCLM - CNN": {
        "ResNet-50": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/ResNet_Results/resnet50_uclm_tl(-3).keras",
            "1e-4": "/content/drive/MyDrive/Ara Proje/saved_models/ResNet_Results/resnet50_uclm_tl(-4).keras",
            "1e-5": "/content/drive/MyDrive/Ara Proje/saved_models/ResNet_Results/resnet50_uclm_tl(-5).keras",
        },
        "EfficientNet": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/UCLM_TL_Results/EfficientNet_UCLM_best_lr_1en03.keras",
            "1e-4": "/content/drive/MyDrive/Ara Proje/saved_models/UCLM_TL_Results/EfficientNet_UCLM_best_lr_1en04.keras",
            "1e-5": "/content/drive/MyDrive/Ara Proje/saved_models/UCLM_TL_Results/EfficientNet_UCLM_best_lr_1en05.keras",
        },
        "NasNet": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/Nasnet_Results/nasnetlarge_uclm_tl(-3).keras",
            "1e-4": "/content/drive/MyDrive/Ara Proje/saved_models/Nasnet_Results/nasnetlarge_uclm_tl(-4).keras",
            "1e-5": "/content/drive/MyDrive/Ara Proje/saved_models/Nasnet_Results/nasnetlarge_uclm_tl(-5).keras",
        },
        "Inception": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/UCLM_TL_Results/InceptionV3_BUSUCLM_best_lr_1en03.keras",
            "1e-4": "/content/drive/MyDrive/Ara Proje/saved_models/UCLM_TL_Results/InceptionV3_BUSUCLM_best_lr_1en04.keras",
            "1e-5": "/content/drive/MyDrive/Ara Proje/saved_models/UCLM_TL_Results/InceptionV3_BUSUCLM_best_lr_1en05.keras",
        },
        "Xception": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/Xception_Results/xception_uclm_tl(-3).keras",
            "1e-4": "/content/drive/MyDrive/Ara Proje/saved_models/Xception_Results/xception_uclm_tl(-4).keras",
            "1e-5": "/content/drive/MyDrive/Ara Proje/saved_models/Xception_Results/xception_uclm_tl(-5).keras",
        },
        "DenseNet": {
            "1e-3": "/content/drive/MyDrive/Ara Proje/saved_models/UCLM_TL_Results/DenseNet121_BUSUCLM_best_lr_1en03.keras",
            "1e-4": "/content/drive/MyDrive/Ara Proje/saved_models/UCLM_TL_Results/DenseNet121_BUSUCLM_best_lr_1en04.keras",
            "1e-5": "/content/drive/MyDrive/Ara Proje/saved_models/UCLM_TL_Results/DenseNet121_BUSUCLM_best_lr_1en05.keras",
        }
    },

    "BUSI - Transformer": {
        "BEiT": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/beit_busi_final_v6.pth",
        "CAiT": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/cait_busi_final_v5.pth",
        "Swin Transformer": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/swin_bus_UCLM.pth",
        "DeiT-Tiny": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/deit_busi_final_v3.pth",
        "ViT (Standard)": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/vit_b16_busi_model.pth"
    },

    "BUS-UCLM - Transformer": {
        "BEiT": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/beit_bus_uclm.pth",
        "CAiT": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/cait_bus_uclm.pth",
        "Swin Transformer": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/swin_bus_UCLM.pth",
        "DeiT-Tiny": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/deit_bus_UCLM.pth",
        "ViT (Standard)": "/content/drive/MyDrive/Ara Proje/saved_models/TF models/vit_standard_uclm.pth",
    },

    "BUS-UCLM - DA %30": {
        "ResNet-50": "/content/drive/MyDrive/Ara Proje/saved_models/ResNet_Results/resnet_DA_30_final.keras",
        "EfficientNet": "/content/drive/MyDrive/Ara Proje/saved_models/EfficientNet_Results/EfficientNet_30_DA.keras",
        "NasNet": "/content/drive/MyDrive/Ara Proje/saved_models/Nasnet_Results/nasnet_DA_30_final.keras",
        "Inception": "/content/drive/MyDrive/Ara Proje/saved_models/domain/30_train/InceptionV3_DA_BUSUCLM_30.keras",
        "Xception": "/content/drive/MyDrive/Ara Proje/saved_models/Xception_Results/xception_DA_30_final.keras",
        "DenseNet": "/content/drive/MyDrive/Ara Proje/saved_models/domain/30_train/DenseNet_DA_BUSUCLM_30.keras",
    },

    "BUS-UCLM - DA %50": {
        "ResNet-50": "/content/drive/MyDrive/Ara Proje/saved_models/ResNet_Results/resnet_DA_50_final.keras",
        "EfficientNet": "/content/drive/MyDrive/Ara Proje/saved_models/EfficientNet_Results/effnet_DA_50_v2_final.keras",
        "NasNet": "/content/drive/MyDrive/Ara Proje/saved_models/Nasnet_Results/nasnet_DA_50_.keras",
        "Inception": "/content/drive/MyDrive/Ara Proje/saved_models/domain/50_train/InceptionV3_DA_BUSUCLM_50.keras",
        "Xception": "/content/drive/MyDrive/Ara Proje/saved_models/Xception_Results/xception_DA_50_final.keras",
        "DenseNet": "/content/drive/MyDrive/Ara Proje/saved_models/domain/50_train/DenseNet_DA_BUSUCLM_50.keras",
    }
}
# ================= CACHE =================
CACHE = {"key": None, "model": None, "type": None}

# ================= LOAD MODEL =================
def get_model(dataset, model_name, lr):
    node = MODEL_REGISTRY[dataset][model_name]
    model_path = node[lr] if isinstance(node, dict) else node
    key = f"{dataset}-{model_name}-{lr}"

    if CACHE["key"] == key:
        return CACHE["model"], CACHE["type"]

    if model_path.endswith(".pth"):
        cfg = TORCH_MODELS[model_name]
        model = timm.create_model(cfg["arch"], pretrained=False, num_classes=3)
        ckpt = torch.load(model_path, map_location="cpu")
        model.load_state_dict(ckpt.get("model_state_dict", ckpt))
        model.eval()
        mtype = "torch"
    else:
        model = tf.keras.models.load_model(model_path)
        mtype = "tf"

    CACHE.update({"key": key, "model": model, "type": mtype})
    return model, mtype

# ================= PREDICT =================
# ================= PREDICT =================
def predict(dataset, model_name, lr, img):
    if img is None:
        return "Görüntü yükle", {}

    img = img.convert("RGB")
    model, mtype = get_model(dataset, model_name, lr)

    # ---- TORCH (Transformer Modelleri) ----
    if mtype == "torch":
        cfg = TORCH_MODELS[model_name]
        tfm = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(cfg["mean"], cfg["std"])
        ])
        x = tfm(img).unsqueeze(0)
        with torch.no_grad():
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[0].numpy()

    # ---- TF (CNN Modelleri) ----
    else:
        # --- ÖZEL BOYUT KONTROLLERİ ---
        if model_name == "NasNet":
            # Saf BUS-UCLM için 331, diğerleri (BUSI, DA vb.) için 224
            size = (331, 331) if dataset == "BUS-UCLM - CNN" else (224, 224)

        elif model_name == "Xception":
            # Saf BUS-UCLM için 299, diğerleri için 224
            size = (299, 299) if dataset == "BUS-UCLM - CNN" else (224, 224)

        else:
            # Diğer tüm modeller (ResNet, Inception vb.) için sözlükteki değeri kullan
            size = TF_INPUT_SIZES[model_name]
        # ------------------------------

        img = img.resize(size)
        x = np.expand_dims(np.array(img)/255.0, axis=0)
        probs = model.predict(x, verbose=0)[0]

    # Sonuçları Hazırla
    pred_idx = int(np.argmax(probs))
    label = f"{CLASSES[pred_idx]} (%{probs[pred_idx]*100:.2f})"
    prob_dict = {CLASSES[i]: float(probs[i]) for i in range(3)}

    return label, prob_dict

# ================= UI =================
with gr.Blocks(
    css="""
    body {background: linear-gradient(135deg,#e0e7ff,#fdf2f8);}
    .left {background:#6366f1;color:white;padding:20px;border-radius:16px;}
    .right {background:white;padding:20px;border-radius:16px;box-shadow:0 10px 30px rgba(0,0,0,.1);}
    """
) as demo:

    gr.Markdown("##Medikal Ultrason Sınıflandırma Sistemi")

    with gr.Row():
        with gr.Column(scale=1, elem_classes="left"):
            dataset_dd = gr.Dropdown(MODEL_REGISTRY.keys(), label="Dataset")
            model_dd = gr.Dropdown(label="Model")
            lr_dd = gr.Dropdown(label="Learning Rate")
            btn = gr.Button("Tahmin Et")

        with gr.Column(scale=2, elem_classes="right"):
            img_input = gr.Image(type="pil", label="Ultrason Görüntüsü")
            result = gr.Textbox(label="Tahmin")
            probs = gr.Label(label="📊 Softmax Olasılıkları (%)")

    dataset_dd.change(
        lambda d: gr.update(choices=MODEL_REGISTRY[d].keys()),
        dataset_dd, model_dd
    )

    model_dd.change(
        lambda d, m: gr.update(
            choices=list(MODEL_REGISTRY[d][m].keys()) if isinstance(MODEL_REGISTRY[d][m], dict) else ["-"]
        ),
        [dataset_dd, model_dd], lr_dd
    )

    btn.click(
        predict,
        [dataset_dd, model_dd, lr_dd, img_input],
        [result, probs]
    )

demo.launch(share=True)


/tmp/ipython-input-2781546559.py:202: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33b5b600088676b47a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
